In [1]:

import keras
import tensorflow as tf
from IPython.core.display import Image
from keras._tf_keras.keras.preprocessing import image
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras import utils, layers
import numpy as np
import os




In [2]:
# Підготовка даних (як у умові)
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

x_train = x_train.reshape(60000, 784).astype('float32') / 255.0
x_test  = x_test.reshape(10000, 784).astype('float32') / 255.0

y_train = utils.to_categorical(y_train, 10)
y_test  = utils.to_categorical(y_test, 10)

print('x_train shape:', x_train.shape, 'y_train shape:', y_train.shape)


x_train shape: (60000, 784) y_train shape: (60000, 10)


In [3]:
def build_model_part3(hp):
    model = keras.Sequential()

    # --- Вхідний шар ---
    model.add(layers.InputLayer(input_shape=(x_train.shape[1],)))

    # --- Прихований шар ---
    model.add(layers.Dense(
        units=hp.Int('units', min_value=16, max_value=128, step=16),
        activation=hp.Choice('hidden_activation', values=['relu', 'sigmoid', 'tanh', 'elu', 'selu'])
    ))

    # --- Вихідний шар (для класифікації) ---
    model.add(layers.Dense(y_train.shape[1], activation=hp.Choice('output_activation', values=['relu', 'sigmoid', 'tanh', 'elu', 'selu'])))

    # --- Оптимізатор як гіперпараметр ---
    optimizer = hp.Choice('optimizer', values=['adam', 'rmsprop', 'SGD'])

    # --- Компіляція ---
    model.compile(
        optimizer=optimizer,
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )

    return model

In [4]:
from kerastuner.tuners import BayesianOptimization

tuner = BayesianOptimization(
    build_model_part3,
    objective='val_accuracy',
    max_trials=8,
    executions_per_trial=1,
    directory='tuner_part3_dir',
    project_name='part3_bayes'
)

tuner.search_space_summary()



Search space summary
Default search space size: 4
units (Int)
{'default': None, 'conditions': [], 'min_value': 16, 'max_value': 128, 'step': 16, 'sampling': 'linear'}
hidden_activation (Choice)
{'default': 'relu', 'conditions': [], 'values': ['relu', 'sigmoid', 'tanh', 'elu', 'selu'], 'ordered': False}
output_activation (Choice)
{'default': 'relu', 'conditions': [], 'values': ['relu', 'sigmoid', 'tanh', 'elu', 'selu'], 'ordered': False}
optimizer (Choice)
{'default': 'adam', 'conditions': [], 'values': ['adam', 'rmsprop', 'SGD'], 'ordered': False}


C:\Users\dasav\AppData\Local\Temp\ipykernel_208\1710515503.py:1: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner.tuners import BayesianOptimization
C:\Users\dasav\AppData\Local\Programs\Python\Python313\Lib\site-packages\keras\src\layers\core\input_layer.py:27: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


In [11]:
tuner.search(x_train,                  # Дані для навчання
             y_train,                  # Вірні відповіді
             batch_size=256,           # Розмір міні-вибірки
             epochs=20,                # Кількість епох навчання
             validation_split=0.2,     # Частина даних, яка буде використовуватися для перевірки
             )

In [12]:
tuner.results_summary()

Results summary
Results in tuner_part3_dir\part3_bayes
Showing 10 best trials
Objective(name="val_accuracy", direction="max")

Trial 0 summary
Hyperparameters:
units: 80
hidden_activation: tanh
output_activation: sigmoid
optimizer: adam
Score: 0.8844166398048401

Trial 4 summary
Hyperparameters:
units: 96
hidden_activation: elu
output_activation: sigmoid
optimizer: adam
Score: 0.8830000162124634

Trial 2 summary
Hyperparameters:
units: 112
hidden_activation: elu
output_activation: sigmoid
optimizer: SGD
Score: 0.8324999809265137

Trial 3 summary
Hyperparameters:
units: 32
hidden_activation: selu
output_activation: relu
optimizer: SGD
Score: 0.6859166622161865

Trial 7 summary
Hyperparameters:
units: 32
hidden_activation: relu
output_activation: elu
optimizer: rmsprop
Score: 0.48625001311302185

Trial 1 summary
Hyperparameters:
units: 112
hidden_activation: tanh
output_activation: elu
optimizer: adam
Score: 0.24124999344348907

Trial 5 summary
Hyperparameters:
units: 96
hidden_activatio

In [7]:
models = tuner.get_best_models(num_models=3)
for model in models:
    model.summary()
    model.evaluate(x_test, y_test)
    print()

C:\Users\dasav\AppData\Local\Programs\Python\Python313\Lib\site-packages\keras\src\saving\saving_lib.py:797: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 10 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 80)             │        62,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 10)             │           810 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 63,610 (248.48 KB)

 Trainable params: 63,610 (248.48 KB)

...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 112)            │        87,920 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 10)             │         1,130 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 89,050 (347.85 KB)

 Trainable params: 89,050 (347.85 KB)

 Non-trainable params: 0 (0.00 B)

313/313 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.8251 - loss: 0.5033



In [8]:
top_model = tuner.get_best_models(num_models=1)
top_model[0].save('models/task3-best.keras')
Image('shirt.jpg', width=150, height=150)
img = image.load_img('shirt.jpg', target_size=(28, 28), color_mode="grayscale")
# Перетворюмо картинку в масив
x = image.img_to_array(img)
# Змінюємо форму масиву в плаский вектор
x = x.reshape(1, 784)
# Інвертуємо зображення
x = 255 - x
# Нормалізуємо зображення
x /= 255

In [9]:
classes = ['футболка', 'брюки', 'светр', 'плаття', 'пальто', 'туфлі', 'сорочка', 'кросівки', 'сумка', 'чоботи']
prediction = top_model[0].predict(x)
prediction = np.argmax(prediction)
print("Номер класу:", prediction)
print("Назва класу:", classes[prediction])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step
Номер класу: 6
Назва класу: сорочка
